In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

# Law Article 4

## cu_4_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 因维护公司价值及股东权益所必需回购股份的 |
| constraint | 应当符合以下条件之一：（一）公司股票收盘价格低于最近一期每股净资产；（二）连续20个交易日内公司股票收盘价格跌幅累计达到20%；（三）公司股票收盘价格低于最近一年股票最高收盘价格的50%；（四）中国证券监督管理委员会（以下简称中国证监会）规定的其他条件。 |
| contextual_info | nan |
| note | 1. 决议通过日当日检查. 简化假设; 股价以收盘价为准, 不考虑日内价格变化
2. 不考虑（四）中国证券监督管理委员会（以下简称中国证监会）规定的其他条件。3. 一年按照250个交易日计算. 4. condition理解为检查'决议通过日'的情况, 计算各种指标也以该日为基准 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2171 |
| completion_tokens | 6544 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_4_1(df):
    '''
    检查cu_4_1的合规性：
    主体：上市公司（所有行均为上市公司）
    条件：决议通过日存在维护公司价值的回购方案
    约束：需满足三个价格条件之一
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_4_1_subject'] = True  # 所有公司均为上市公司
    df['cu_4_1_condition'] = False
    df['cu_4_1_constraint'] = False
    
    # 1. 验证触发条件（仅在决议通过日检查）
    condition_mask = (
        (df['日期'] == df['决议通过日']) &
        df['存在回购方案'] &
        (df['回购用途'] == '维护公司价值及股东权益所必需')
    )
    df.loc[condition_mask, 'cu_4_1_condition'] = True
    
    # 2. 计算约束条件（所有行独立计算）
    # 条件一：收盘价低于每股净资产
    cond1 = df['收盘价'] < df['每股净资产']
    
    # 条件二：20交易日累计跌幅超20%
    df['20d_prior_close'] = df.groupby('公司简称')['收盘价'].shift(19)
    cond2 = ((df['收盘价'] - df['20d_prior_close']) / df['20d_prior_close'] <= -0.2).fillna(False)
    
    # 条件三：股价低于年高点的50%
    df['250d_max'] = df.groupby('公司简称')['收盘价'].transform(lambda x: x.rolling(250, min_periods=1).max())
    cond3 = df['收盘价'] <= df['250d_max'] * 0.5
    
    # 综合约束条件
    df['cu_4_1_constraint'] = cond1 | cond2 | cond3
    
    # 清理中间列
    df.drop(columns=['20d_prior_close', '250d_max'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_cu_4_1(df)
df

---

# Law Article 13

## cu_13_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施竞价回购 |
| constraint | 应当符合公司股票上市已满6个月 |
| contextual_info | nan |
| note | 6个月按照自然月计算 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2033 |
| completion_tokens | 1965 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_13_1(df):
    '''
    检查cu_13_1的合规性：
    主体：上市公司
    条件：实施竞价回购
    约束：股票上市已满6个自然月
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_13_1_subject'] = True  # 所有公司均为上市公司
    df['cu_13_1_condition'] = False
    df['cu_13_1_constraint'] = False
    
    # 1. 验证触发条件：实施竞价回购
    # 需要同时满足存在回购方案且回购方式为竞价回购
    is_auction_repurchase = (df['回购方式'] == '竞价回购') & df['存在回购方案']
    df.loc[is_auction_repurchase, 'cu_13_1_condition'] = True
    
    # 2. 验证约束条件：上市已满6个自然月
    # 计算上市日期+6个月的自然月
    six_months_later = df['上市日期'] + pd.DateOffset(months=6)
    # 判断当前日期是否满足上市满6个月
    df['cu_13_1_constraint'] = df['日期'] >= six_months_later
    
    return df

In [ ]:
df = check_cu_13_1(df)
df

---

## cu_13_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施竞价回购 |
| constraint | 应当符合公司最近1年无重大违法行为 |
| contextual_info | nan |
| note | 年按照自然年计算 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2032 |
| completion_tokens | 4096 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_13_2(df):
    '''
    检查cu_13_2的合规性：
    主体：上市公司
    条件：实施竞价回购
    约束：最近1个自然年无重大违法行为
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_13_2_subject'] = True  # 所有公司均为上市公司
    df['cu_13_2_condition'] = False
    df['cu_13_2_constraint'] = None

    # 1. 责任主体验证（所有行自动满足）
    
    # 2. 触发条件验证
    df['cu_13_2_condition'] = df['存在回购方案'] & (df['回购方式'] == '竞价回购')

    # 3. 约束条件验证（最近自然年无重大违法行为）
    # 创建违规年份记录表
    violation_mask = df['公司法律情况'] == '重大违法行为'
    violation_df = df.loc[violation_mask, ['公司简称', '日期']].copy()
    violation_df['violation_year'] = violation_df['日期'].dt.year
    company_violation_years = violation_df.groupby(['公司简称', 'violation_year']).size().reset_index()
    company_violation_years['has_violation'] = True

    # 计算当前年份和前一年份
    df['current_year'] = df['日期'].dt.year
    df['prev_year'] = df['current_year'] - 1

    # 合并违规记录
    df = df.merge(
        company_violation_years.rename(columns={'violation_year': 'prev_year'}),
        on=['公司简称', 'prev_year'],
        how='left'
    )
    
    # 标记约束条件
    df['cu_13_2_constraint'] = ~df['has_violation'].fillna(False)

    # 清理临时列
    df.drop(columns=['current_year', 'prev_year', 'violation_year', 'has_violation'], 
            inplace=True, errors='ignore')

    return df

In [ ]:
df = check_cu_13_2(df)
df

---

## cu_13_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 因触及本指引第四条规定条件而启动回购并减少注册资本 |
| constraint | 不适用股票上市已满6个月的要求 |
| contextual_info | nan |
| note | constraint项保留为None不做处理; 不考虑减少注册资本的情况; 触及本指引第四条理解为检查'决议通过日'的情况, 计算各种指标也以该日为基准 |
| relation | refer_to |
| target | MEU_4_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2117 |
| completion_tokens | 4520 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_13_7(df):
    '''
    检查cu_13_7的合规性：
    主体：上市公司
    条件：因触及指引第四条条件启动回购（决议通过日股价低于每股净资产或净利润）
    约束：不适用股票上市满6个月要求（保留为None）
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_13_7_subject'] = True  # 所有公司都是上市公司
    df['cu_13_7_condition'] = False
    df['cu_13_7_constraint'] = None  # 根据要求保留为None
    
    # 筛选存在回购方案的行
    mask_repo = df['存在回购方案']
    if mask_repo.any():
        # 获取所有决议通过日及其对应的财务指标
        resolution_dates = df.loc[mask_repo, ['公司简称', '决议通过日']].drop_duplicates()
        # 合并决议通过日当天的数据
        resolution_data = pd.merge(
            resolution_dates,
            df[['公司简称', '日期', '收盘价减每股净资产', '收盘价减每股净利润']],
            left_on=['公司简称', '决议通过日'],
            right_on=['公司简称', '日期'],
            how='left'
        )
        # 判断是否满足条件
        resolution_data['condition_met'] = (
            (resolution_data['收盘价减每股净资产'] <= 0) | 
            (resolution_data['收盘价减每股净利润'] <= 0)
        )
        # 合并回原数据框
        df = df.merge(
            resolution_data[['公司简称', '决议通过日', 'condition_met']],
            on=['公司简称', '决议通过日'],
            how='left'
        )
        # 标记条件为True的行
        df.loc[mask_repo & df['condition_met'], 'cu_13_7_condition'] = True
        # 清理临时列
        df.drop(columns=['condition_met'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_cu_13_7(df)
df

---

# Law Article 15

## cu_15_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | nan |
| constraint | 在回购股份方案中明确拟回购股份数量或者资金总额的上下限，且下限不得低于上限的50% |
| contextual_info | nan |
| note | constraint项检查所有存在回购方案的行 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2047 |
| completion_tokens | 8588 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_15_2(df):
    '''
    检查cu_15_2的合规性：
    主体：上市公司（所有行均符合）
    条件：无触发条件
    约束：存在回购方案时需满足数量/资金上下限要求
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_15_2_subject'] = True  # 所有公司均为上市公司
    df['cu_15_2_condition'] = True  # 无触发条件
    df['cu_15_2_constraint'] = True  # 默认标记为合规

    # 仅处理存在回购方案的行
    mask = df['存在回购方案']
    if mask.any():
        # 计算指标存在性
        has_quantity = (~df['回购数量上限'].isna()) & (~df['回购数量下限'].isna())
        has_funds = (~df['资金总额上限'].isna()) & (~df['资金总额下限'].isna())
        
        # 计算单项合规性
        quantity_valid = (df['回购数量下限'] >= 0.5 * df['回购数量上限'])
        funds_valid = (df['资金总额下限'] >= 0.5 * df['资金总额上限'])
        
        # 复合校验逻辑（必须满足所有存在的指标）
        constraint_check = (
            # 至少存在一个指标
            (has_quantity | has_funds) &
            # 存在的指标均需合规
            quantity_valid.where(has_quantity, True) &
            funds_valid.where(has_funds, True)
        )
        
        # 更新约束标记
        df.loc[mask, 'cu_15_2_constraint'] = constraint_check[mask]

    return df

In [ ]:
df = check_cu_15_2(df)
df

---

# Law Article 16

## cu_16_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 进行竞价回购 |
| constraint | 价格上限原则上不应高于董事会审议通过回购股份决议前30个交易日（不含停牌日）交易均价的200% |
| contextual_info | nan |
| note | 不考虑"原则上"的松弛条件; 每日股价以收盘价为准, 不考虑日内价格变化; 不存在这30个交易日无法取到的情况; 对每个公司每个回购计划期间的['决议通过日', '实施截止日']之间的日期标记constraint, 不符合这些条件的行可保留为None; 价格是否突破上限按照['申报价格']计算 |
| relation | refer_to |
| target | MEU_75_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2166 |
| completion_tokens | 4911 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_16_1(df):
    '''
    检查cu_16_1的合规性：
    主体：上市公司
    条件：进行竞价回购
    约束：申报价格不超过决议通过日前30个交易日均价的200%
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_16_1_subject'] = True
    df['cu_16_1_condition'] = (df['回购方式'] == '竞价回购') & df['存在回购方案']
    df['cu_16_1_constraint'] = None
    
    # 提取有效回购计划
    valid_plans = df[df['cu_16_1_condition']][['公司简称', '决议通过日', '实施开始日', '实施截止日']].drop_duplicates()
    
    if not valid_plans.empty:
        # 计算价格上限
        def calculate_limit(row):
            company_data = df[(df['公司简称'] == row['公司简称']) & 
                             (df['日期'] < row['决议通过日'])]
            last_30 = company_data.nlargest(30, '日期')
            return last_30['收盘价'].mean() * 2
        
        valid_plans['price_limit'] = valid_plans.apply(calculate_limit, axis=1)
        
        # 合并价格限制
        df = df.merge(
            valid_plans,
            on=['公司简称', '决议通过日', '实施开始日', '实施截止日'],
            how='left'
        )
        
        # 标记约束条件
        in_period_mask = (
            (df['日期'].between(df['实施开始日'], df['实施截止日'])) &
            df['cu_16_1_condition']
        )
        df.loc[in_period_mask, 'cu_16_1_constraint'] = (
            df.loc[in_period_mask, '申报价格'] <= df.loc[in_period_mask, 'price_limit']
        )
        
        # 清理临时列
        df.drop(columns=['price_limit'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_cu_16_1(df)
df

---

# Law Article 18

## cu_18_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 进行股份回购的申报 |
| constraint | 申报价格不得为公司股票当日交易涨幅限制的价格 |
| contextual_info | nan |
| note | 1. 假设当日涨跌幅限制为前日收盘价的正负10%; 2. condition理解为检查'决议通过日'的情况, 计算各种指标也以该日为基准 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2069 |
| completion_tokens | 2654 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_18_7(df):
    '''
    检查cu_18_7的合规性：
    主体：上市公司
    条件：在股份回购决议通过日进行申报
    约束：申报价格不得为当日涨跌幅限制价格
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_18_7_subject'] = True  # 所有公司均为上市公司
    df['cu_18_7_condition'] = False
    df['cu_18_7_constraint'] = None

    # 1. 验证触发条件（决议通过日且存在回购方案）
    condition_mask = (
        (df['日期'] == df['决议通过日']) &  # 判断是否为决议通过当日
        df['存在回购方案']  # 确认存在有效回购方案
    )
    df.loc[condition_mask, 'cu_18_7_condition'] = True

    # 2. 验证约束条件（仅处理符合条件的数据）
    if df[condition_mask].shape[0] > 0:
        # 计算涨跌幅限制价格
        mask = df['cu_18_7_condition']
        prev_close = df.loc[mask, '前收盘价']
        upper_limit = prev_close * 1.1  # 涨停价
        lower_limit = prev_close * 0.9  # 跌停价
        
        # 获取申报价格并进行比对
        apply_price = df.loc[mask, '申报价格']
        price_constraint = ~apply_price.isin([upper_limit, lower_limit])
        
        # 标记约束合规性
        df.loc[mask, 'cu_18_7_constraint'] = price_constraint.astype(bool)

    return df

In [ ]:
df = check_cu_18_7(df)
df

---

# Law Article 19

## cu_19_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施竞价回购 |
| constraint | 实施期限不超过12个月 |
| contextual_info | 自董事会或股东大会（如须）审议通过回购股份决议之日起算 |
| note | 对存在回购方案的所有交易日进行检查; 12个月按照自然月计算 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2053 |
| completion_tokens | 6351 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_19_1(df):
    '''
    检查cu_19_1的合规性：
    主体：上市公司
    条件：实施竞价回购
    约束：实施期限不超过12个月（自然月）
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_19_1_subject'] = True  # 所有公司均为上市公司
    df['cu_19_1_condition'] = False
    df['cu_19_1_constraint'] = None
    
    # 1. 验证触发条件：存在回购方案且为竞价方式
    condition_mask = (df['存在回购方案']) & (df['回购方式'] == '竞价回购')
    df.loc[condition_mask, 'cu_19_1_condition'] = True
    
    # 2. 验证约束条件（仅对符合条件的行）
    valid_rows = df[condition_mask].index
    if not valid_rows.empty:
        # 计算自然月期限
        resolution_date = df.loc[valid_rows, '决议通过日']
        deadline_date = resolution_date + pd.DateOffset(months=12)
        
        # 比较实施截止日与期限
        df.loc[valid_rows, 'cu_19_1_constraint'] = (
            df.loc[valid_rows, '实施截止日'] <= deadline_date
        )
    
    return df

In [ ]:
df = check_cu_19_1(df)
df

---

## cu_19_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 因维护公司价值及股东权益所必需回购股份的，实施竞价回购 |
| constraint | 回购实施期限自股东大会或者董事会审议通过最终回购股份方案之日起不超过3个月 |
| contextual_info | nan |
| note | 采用数据中的'审议通过日'即可, 不用考虑是股东大会还是董事会审议; 3个月按自然月计算 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2074 |
| completion_tokens | 6412 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_19_2(df):
    '''
    检查cu_19_2的合规性：
    主体：上市公司
    条件：因维护公司价值实施竞价回购
    约束：回购实施期不超过审议通过后3个月
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_19_2_subject'] = True  # 所有公司均为上市公司
    df['cu_19_2_condition'] = False
    df['cu_19_2_constraint'] = False
    
    # 1. 验证责任主体（所有行自动满足）
    
    # 2. 验证触发条件
    condition_mask = (
        df['存在回购方案'] &
        (df['回购用途'] == '维护公司价值及股东权益所必需') &
        (df['回购方式'] == '竞价回购')
    )
    df.loc[condition_mask, 'cu_19_2_condition'] = True
    
    # 3. 验证约束条件（自然月计算）
    df['cu_19_2_constraint'] = (
        df['实施截止日'] <= df['决议通过日'] + pd.DateOffset(months=3)
    )
    
    # 处理日期空值情况
    df['cu_19_2_constraint'] = df['cu_19_2_constraint'].fillna(False)
    
    return df

In [ ]:
df = check_cu_19_2(df)
df

---

# Law Article 21

## cu_21_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 回购股份用于股权激励或者员工持股计划、转换上市公司发行的可转换为股票的公司债券、维护公司价值及股东权益所必需的 |
| constraint | 合计持有的本公司股份数不得超过本公司已发行股份总额的10% |
| contextual_info | nan |
| note | 合计持有'理解为累计回购比例 |
| relation | exclude |
| target | MEU_9_3 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2063 |
| completion_tokens | 3139 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_21_2(df):
    """
    检查cu_21_2的合规性：
    主体：上市公司（所有行自动满足）
    条件：存在回购方案且用途符合法定情形
    约束：累计回购比例不超过总股本的10%
    """
    df = df.copy()
    
    # 1. 验证责任主体（所有公司均为上市公司）
    df['cu_21_2_subject'] = True
    
    # 2. 验证触发条件
    valid_condition = (
        df['存在回购方案'] & 
        df['回购用途'].isin([
            '股权激励或者员工持股计划',
            '转换上市公司发行的可转换为股票的公司债券',
            '维护公司价值及股东权益所必需的'
        ])
    )
    df['cu_21_2_condition'] = valid_condition
    
    # 3. 验证约束条件（所有行独立检查）
    # 计算累计回购比例（处理除零情况）
    total_shares = df['总股本'].replace(0, pd.NA)  # 防止除零错误
    df['累计回购比例'] = df['累计回购数量'].astype(float) / total_shares.astype(float)
    
    # 判断是否超过10%阈值
    df['cu_21_2_constraint'] = df['累计回购比例'].fillna(0) <= 0.1
    
    # 清理中间列
    df.drop(columns=['累计回购比例'], inplace=True)
    
    return df

In [ ]:
df = check_cu_21_2(df)
df

---

# Law Article 42

## cu_42_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 因维护公司价值及股东权益所必需而回购股份的，处于上市公司定期报告、业绩预告或者业绩快报披露前10个交易日内（因特殊原因推迟定期报告披露日期的，自原预约公告日前10个交易日起算，至披露前1个交易日） |
| constraint | 不得采用集中竞价交易方式出售股份 |
| contextual_info | nan |
| note | 不考虑报告推迟情况; 只考虑同时进行的回购计划的回购方式, 不考虑前后对应关系 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2103 |
| completion_tokens | 8978 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_42_2(df):
    '''
    检查cu_42_2合规性：
    主体：上市公司
    条件：维护价值回购且处于定期报告披露前10日
    约束：不得集中竞价出售
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_42_2_subject'] = True  # 所有行均为上市公司
    df['cu_42_2_condition'] = False
    df['cu_42_2_constraint'] = None

    # 生成交易日历参考系（按公司分组）
    company_calendars = df.groupby('公司简称')['日期'].apply(lambda x: sorted(x.unique()))

    # 构建披露日期前10日窗口（考虑多公告类型）
    df['in_window'] = False
    for company, dates in company_calendars.items():
        # 获取该公司所有定期类公告日期
        announcements = df[(df['公司简称'] == company) & 
                          (df['公告类型'].isin(['定期报告','业绩预告','业绩快报']))]['公告日期'].unique()
        
        # 建立日期索引映射
        date_index = {date: idx for idx, date in enumerate(dates)}
        
        # 标记所有窗口日期
        window_dates = set()
        for d in announcements:
            if d not in date_index:
                continue
            start_idx = max(0, date_index[d] - 10)
            window_dates.update(dates[start_idx:date_index[d]])
        
        # 更新标记列
        df.loc[(df['公司简称'] == company) & (df['日期'].isin(window_dates)), 'in_window'] = True

    # 条件验证（回购用途+时间窗口）
    condition_mask = (
        df['存在回购方案'] & 
        (df['回购用途'] == '维护公司价值及股东权益所必需') &
        df['in_window']
    )
    df['cu_42_2_condition'] = condition_mask

    # 约束验证（仅检查符合前两项的行）
    constraint_mask = condition_mask & df['cu_42_2_subject']
    # 假设存在出售计划即视为集中竞价方式（因数据无明确字段）
    df.loc[constraint_mask, 'cu_42_2_constraint'] = ~df.loc[constraint_mask, '存在出售计划']

    # 清理中间列
    df.drop(columns=['in_window'], inplace=True)

    return df

In [ ]:
df = check_cu_42_2(df)
df

---

# Law Article 45

## cu_45_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 采用集中竞价交易方式出售已回购股份 |
| constraint | 申报价格不得为公司股票当日交易跌幅限制的价格 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2034 |
| completion_tokens | 4323 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_45_2(df):
    '''
    检查cu_45_2的合规性：
    主体：上市公司
    条件：采用集中竞价交易方式出售已回购股份
    约束：申报价格不得为当日跌停价
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_45_2_subject'] = True  # 所有公司均为上市公司
    df['cu_45_2_condition'] = df['存在出售计划']
    df['cu_45_2_constraint'] = None
    
    # 计算当日跌停价（前收盘价的90%）
    df['跌停价'] = df['前收盘价'] * 0.9
    
    # 仅对符合主体和条件的行进行约束检查
    valid_mask = df['cu_45_2_subject'] & df['cu_45_2_condition']
    
    # 检查申报价格是否等于跌停价（包含NaN处理）
    df.loc[valid_mask, 'cu_45_2_constraint'] = ~(
        df.loc[valid_mask, '申报价格'].eq(df.loc[valid_mask, '跌停价']) |
        df.loc[valid_mask, '申报价格'].isna()  # 申报价格缺失视为违规
    )
    
    # 清理中间列
    df.drop(columns=['跌停价'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_cu_45_2(df)
df

---

## cu_45_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 采用集中竞价交易方式出售已回购股份且每日出售数量超过10万股 |
| constraint | 每日出售的数量不得超过出售计划披露日前20个交易日日均盘中成交量的25% |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2048 |
| completion_tokens | 8598 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd
import numpy as np

def check_cu_45_3(df):
    '''
    检查cu_45_3的合规性：
    主体：上市公司
    条件：采用集中竞价交易方式出售已回购股份且每日出售数量超过10万股
    约束：每日出售数量不得超过出售计划披露日前20个交易日日均成交量的25%
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_45_3_subject'] = True  # 所有公司都是上市公司
    df['cu_45_3_condition'] = False
    df['cu_45_3_constraint'] = False
    
    # 2. 验证触发条件：存在出售计划且当日出售数量超过10万股
    # 假设存在出售计划即表示采用集中竞价方式
    condition_mask = (df['存在出售计划']) & (df['当日出售数量'] > 100000)
    df.loc[condition_mask, 'cu_45_3_condition'] = True
    
    # 3. 验证约束条件：计算阈值并比较
    # 获取所有出售计划的披露日
    plan_disclosures = df[df['存在出售计划']][['公司简称', '出售计划披露日']].drop_duplicates()
    
    thresholds = []
    for _, row in plan_disclosures.iterrows():
        company = row['公司简称']
        disclosure_date = row['出售计划披露日']
        
        # 获取该公司在披露日前的所有交易日
        company_mask = (df['公司简称'] == company) & (df['日期'] < disclosure_date)
        pre_dates = df[company_mask]['日期'].unique()
        pre_dates_sorted = sorted(pre_dates, reverse=True)
        
        # 取前20个交易日
        if len(pre_dates_sorted) >= 20:
            selected_dates = pre_dates_sorted[:20]
        else:
            selected_dates = pre_dates_sorted  # 不足20天则取全部可用
        
        # 计算这些日期的平均成交量
        avg_volume = df[(df['公司简称'] == company) & (df['日期'].isin(selected_dates))]['成交量'].mean()
        threshold = avg_volume * 0.25 if not np.isnan(avg_volume) else np.nan
        thresholds.append({
            '公司简称': company,
            '出售计划披露日': disclosure_date,
            'threshold': threshold
        })
    
    if thresholds:
        thresholds_df = pd.DataFrame(thresholds)
        df = df.merge(thresholds_df, on=['公司简称', '出售计划披露日'], how='left')
        
        # 标记约束条件：当日出售数量 <= threshold，且存在出售计划
        constraint_mask = df['存在出售计划'] & (~df['threshold'].isna())
        df.loc[constraint_mask, 'cu_45_3_constraint'] = df.loc[constraint_mask, '当日出售数量'] <= df.loc[constraint_mask, 'threshold']
    else:
        df['threshold'] = np.nan
    
    # 清理临时列
    df.drop(columns=['threshold'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_cu_45_3(df)
df

---

## cu_45_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 采用集中竞价交易方式出售已回购股份 |
| constraint | 在任意连续90日内，出售股份的总数不得超过公司股份总数的1% |
| contextual_info | nan |
| note | 90日以自然日计算 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2046 |
| completion_tokens | 7788 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_45_4(df):
    '''
    检查cu_45_4合规性：
    主体：上市公司（全部标记为True）
    条件：存在集中竞价出售已回购股份计划
    约束：90自然日内累计出售≤总股本1%
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_45_4_subject'] = True
    df['cu_45_4_condition'] = df['存在出售计划']
    df['cu_45_4_constraint'] = None

    # 计算自然日窗口累计出售量
    sorted_df = df.sort_values(['公司简称', '日期']).copy()
    sorted_df.set_index('日期', inplace=True)
    
    # 使用自然日滚动窗口计算累计出售量
    sorted_df['累计出售数量_90d'] = sorted_df.groupby('公司简称', group_keys=False)['当日出售数量'] \
        .rolling('90D', min_periods=0).sum().reset_index(level=0, drop=True)
    
    sorted_df.reset_index(inplace=True)
    
    # 合并计算结果到原始数据
    df = df.merge(
        sorted_df[['公司简称', '日期', '累计出售数量_90d']],
        on=['公司简称', '日期'],
        how='left'
    )

    # 计算约束条件（总股本取当日值）
    df['cu_45_4_constraint'] = df['累计出售数量_90d'] <= df['总股本'] * 0.01
    
    # 处理空值（无出售时默认合规）
    df['cu_45_4_constraint'] = df['cu_45_4_constraint'].fillna(True)

    return df

In [ ]:
df = check_cu_45_4(df)
df

---

# Law Article 46

## cu_46_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 采用集中竞价交易方式出售已回购股份期间 |
| constraint | 首次出售已回购股份事实发生后应当在次1个交易日内披露出售进展情况公告 |
| contextual_info | nan |
| note | 对每一个需要披露的日期标记constraint情况, 对不需要的日期保留为None |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2062 |
| completion_tokens | 9422 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd
import numpy as np

def check_cu_46_2(df):
    '''
    检查cu_46_2的合规性：
    主体：上市公司（所有行自动满足）
    条件：处于集中竞价出售已回购股份期间
    约束：首次出售次交易日披露进展
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_46_2_subject'] = True  # 所有公司均为上市公司
    df['cu_46_2_condition'] = False
    df['cu_46_2_constraint'] = None
    
    # 2. 条件标记（集中竞价出售期间）
    valid_condition = (
        df['存在出售计划'] 
        & (df['日期'] >= df['出售开始日']) 
        & (df['日期'] <= df['出售截止日'])
    )
    df['cu_46_2_condition'] = valid_condition
    
    # 3. 约束标记（首次出售次交易日披露）
    if (df['当日出售数量'] > 0).any():
        # 获取各公司首次出售日
        first_sale = df[df['当日出售数量'] > 0].groupby('公司简称')['日期'].min().reset_index()
        first_sale.columns = ['公司简称', 'first_sale_date']
        
        # 获取各公司交易日序列
        trading_dates = df.groupby('公司简称')['日期'].apply(lambda x: x.sort_values().unique()).reset_index()
        trading_dates.columns = ['公司简称', 'date_seq']
        
        # 计算次交易日
        merged = trading_dates.merge(first_sale, how='left', on='公司简称')
        merged['next_trading_date'] = merged.apply(
            lambda x: x['date_seq'][np.searchsorted(x['date_seq'], x['first_sale_date']) + 1] 
            if (pd.notnull(x['first_sale_date']) and 
                (np.searchsorted(x['date_seq'], x['first_sale_date']) + 1 < len(x['date_seq']))) 
            else pd.NaT,
            axis=1
        )
        
        # 合并次交易日到原始数据
        df = df.merge(merged[['公司简称', 'next_trading_date']], on='公司简称', how='left')
        
        # 标记约束合规性
        is_disclosure_day = df['日期'] == df['next_trading_date']
        df.loc[is_disclosure_day, 'cu_46_2_constraint'] = df.loc[is_disclosure_day, '披露出售进展'].fillna(False)
        
        df.drop(columns=['next_trading_date'], inplace=True)
    
    return df

In [ ]:
df = check_cu_46_2(df)
df

---

## cu_46_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 采用集中竞价交易方式出售已回购股份期间 |
| constraint | 出售已回购股份占上市公司总股本的比例增加1%, 应当在事实发生之日起3个交易日内披露出售进展情况公告 |
| contextual_info | nan |
| note | 理解为"每增加1%"需要披露; 划分披露的窗口期, 对整个窗口期标记constraint是否符合, 不在窗口期内的可标记为True或者保留为None |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2089 |
| completion_tokens | 11841 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_46_3(df):
    '''
    检查cu_46_3的合规性：
    主体：上市公司
    条件：采用集中竞价交易方式出售已回购股份期间
    约束：每增加1%需在3个交易日内披露
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_46_3_subject'] = True  # 所有公司均为上市公司
    df['cu_46_3_condition'] = False
    df['cu_46_3_constraint'] = None
    
    # 1. 标记触发条件
    condition_mask = (
        df['存在出售计划'] &
        df['日期'].between(df['出售开始日'], df['出售截止日']) &
        df['出售开始日'].notna() &
        df['出售截止日'].notna()
    )
    df.loc[condition_mask, 'cu_46_3_condition'] = True
    
    # 2. 核心逻辑：处理约束条件
    # 仅处理满足主体和条件的记录
    mask = df['cu_46_3_subject'] & df['cu_46_3_condition']
    working_df = df[mask].copy()
    
    # 按公司分组处理
    for (company,), group in working_df.groupby(['公司简称']):
        group = group.sort_values('日期')
        dates = group['日期'].tolist()
        date_to_idx = {date: idx for idx, date in enumerate(dates)}
        
        # 生成触发点队列
        triggers = []
        prev_trigger = 0.0
        for _, row in group.iterrows():
            current_ratio = row['累计出售比例']
            while current_ratio >= prev_trigger + 0.01:
                triggers.append(row['日期'])
                prev_trigger += 0.01
        
        # 处理每个触发点的窗口期
        for trigger_date in triggers:
            if trigger_date not in date_to_idx:
                continue
                
            # 确定窗口期（包含触发日及后续两个交易日）
            trigger_idx = date_to_idx[trigger_date]
            window_indices = range(trigger_idx, min(trigger_idx+3, len(dates)))
            window_dates = [dates[i] for i in window_indices]
            
            # 检查窗口期内是否有披露
            has_disclosure = group[group['日期'].isin(window_dates)]['披露出售进展'].any()
            
            # 更新主数据框
            company_mask = (
                (df['公司简称'] == company) & 
                (df['日期'].isin(window_dates)) & 
                mask
            )
            df.loc[company_mask, 'cu_46_3_constraint'] = has_disclosure
        
        # 标记非窗口期内的合规记录为True
        all_dates_in_condition = group['日期'].unique()
        window_dates_all = []
        for trigger_date in triggers:
            if trigger_date not in date_to_idx:
                continue
            trigger_idx = date_to_idx[trigger_date]
            window_indices = range(trigger_idx, min(trigger_idx+3, len(dates)))
            window_dates_all.extend([dates[i] for i in window_indices])
        
        non_window_dates = list(set(all_dates_in_condition) - set(window_dates_all))
        df.loc[
            (df['公司简称'] == company) & 
            (df['日期'].isin(non_window_dates)) & 
            mask, 
            'cu_46_3_constraint'
        ] = True
    
    # 未触发检查的记录保持None
    return df

In [ ]:
df = check_cu_46_3(df)
df

---

# Law Article 50

## cu_50_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施要约回购 |
| constraint | 要约期限不得少于30个自然日 |
| contextual_info | nan |
| note | 只检查'决议通过日'当日的情况 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2035 |
| completion_tokens | 7712 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_50_2(df):
    '''
    检查cu_50_2的合规性：
    主体：上市公司
    条件：在决议通过日实施要约回购
    约束：要约期限≥30自然日
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_50_2_subject'] = True  # 所有公司均为上市公司
    df['cu_50_2_condition'] = False
    df['cu_50_2_constraint'] = False

    # 1. 责任主体验证（所有行自动满足）
    # 根据问题说明所有公司均为上市公司，无需额外处理

    # 2. 触发条件验证（仅在决议通过日检查）
    condition_mask = (
        (df['日期'] == df['决议通过日']) &  # 当日为决议通过日
        df['存在回购方案'] &  # 存在有效回购方案
        (df['回购方式'] == '要约回购')  # 回购方式为要约回购
    )
    df.loc[condition_mask, 'cu_50_2_condition'] = True

    # 3. 约束条件验证（所有行独立检查）
    # 根据任务要求独立标记，但实际法律逻辑应仅在决议通过日检查
    constraint_mask = (
        (df['日期'] == df['决议通过日']) &  # 仅检查决议通过日
        (df['要约期限'] >= 30)  # 自然日期限要求
    )
    df.loc[constraint_mask, 'cu_50_2_constraint'] = True

    return df

In [ ]:
df = check_cu_50_2(df)
df

---

## cu_50_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施要约回购 |
| constraint | 要约期限不得超过60个自然日 |
| contextual_info | nan |
| note | 只检查'决议通过日'当日的情况 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2034 |
| completion_tokens | 3749 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_50_3(df):
    '''
    检查cu_50_3的合规性：
    主体：上市公司
    条件：在决议通过日实施要约回购
    约束：要约期限不超过60个自然日
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_50_3_subject'] = True  # 所有公司均为上市公司
    df['cu_50_3_condition'] = False
    df['cu_50_3_constraint'] = None
    
    # 1. 验证触发条件（决议通过日当天的要约回购）
    condition_mask = (
        (df['日期'] == df['决议通过日']) &  # 当日为决议通过日
        df['存在回购方案'] &  # 存在有效回购方案
        (df['回购方式'] == '要约回购')  # 回购方式为要约回购
    )
    df.loc[condition_mask, 'cu_50_3_condition'] = True
    
    # 2. 验证约束条件（仅对符合条件的情形进行检查）
    constraint_mask = condition_mask & ~df['要约期限'].isna()
    df.loc[constraint_mask, 'cu_50_3_constraint'] = (
        df.loc[constraint_mask, '要约期限'] <= 60
    )
    
    return df

In [ ]:
df = check_cu_50_3(df)
df

---

# Law Article 51

## cu_51_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施要约回购 |
| constraint | 要约价格不得低于回购股份方案公告日前30个交易日该种股票每日加权平均价的算术平均值 |
| contextual_info | nan |
| note | 方案公告日按照'决议通过日'计算, 只检查当日情况, 价格的计算也以该日为基准; 计算均价时采用收盘价代表该日价格, 不考虑日内价格变化. 不考虑取不到30个交易日的情况, 一定能取到.  |
| relation | refer_to |
| target | Law_75 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2193 |
| completion_tokens | 4877 |


In [ ]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

### 代码实现

In [ ]:
import pandas as pd

def check_cu_51_2(df):
    '''
    检查cu_51_2的合规性：
    主体：上市公司
    条件：实施要约回购
    约束：要约价格≥方案公告日前30个交易日收盘价均值
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_51_2_subject'] = True  # 所有公司均为上市公司
    df['cu_51_2_condition'] = (df['回购方式'] == '要约回购') & df['存在回购方案']
    df['cu_51_2_constraint'] = False
    
    # 预处理：获取有效决议通过日
    valid_resolutions = df[['公司简称', '决议通过日']].dropna().drop_duplicates()
    
    # 计算每个决议通过日前30个交易日的收盘价均值
    avg_prices = []
    for _, row in valid_resolutions.iterrows():
        company = row['公司简称']
        resolution_date = row['决议通过日']
        
        # 获取该公司历史数据
        company_data = df[df['公司简称'] == company].sort_values('日期')
        
        # 筛选决议通过日前30个交易日（不含当日）
        mask = (company_data['日期'] < resolution_date)
        historical_data = company_data[mask].tail(30)
        
        if not historical_data.empty:
            avg_price = historical_data['收盘价'].mean()
            avg_prices.append({
                '公司简称': company,
                '决议通过日': resolution_date,
                'avg_30d_close': avg_price
            })
    
    # 合并平均价数据
    if avg_prices:
        avg_df = pd.DataFrame(avg_prices)
        df = df.merge(avg_df, on=['公司简称', '决议通过日'], how='left')
        
        # 标记约束条件（仅当存在有效数据时）
        valid_mask = (~df['avg_30d_close'].isna()) & (~df['要约价格'].isna())
        df.loc[valid_mask, 'cu_51_2_constraint'] = df.loc[valid_mask, '要约价格'] >= df.loc[valid_mask, 'avg_30d_close']
        df.drop(columns=['avg_30d_close'], inplace=True)
    
    return df

In [ ]:
df = check_cu_51_2(df)
df

---